## Connect to ICW:

In [1]:
import cadspy

In [2]:
username = str(input('User number'))
icw = cadspy.DatabaseConnection(system='ICW', user=username)

User number u243700
Enter Password:  ··············


<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:salmon;font-family:Verdana,sans-serif;font-size:16px;">

<font size="4">Can't connect to ICW? Instructions on how to get access to it in the links below:

</font>
</div>

- Step 1: [Requesting Access to GitHub](https://baplc.sharepoint.com/sites/ask/SitePages/Requesting-Access-to-GitHub.aspx)

Once your access to GitHub has been aproved, you need to:

- Step 2: [Request access to British-Ent GitHub organisation](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/Corporate_Directory_git.md)

And

- Step 3: [Setup Sagemaker Studio to access GitHub](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/sagemaker_to_github_setup.md)

<br>

Below are some packages to get you started. You don't have to use them but you may find them useful!

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
# diplay all rows and cols when using 'dataframe'.head() or 'dataframe'.tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<br>

### Data

#### S19 Lounge Eligibility data

In [ ]:
query = """

sel * from LDB_SBOX_OR.HACKATHON_OPS_LOUNGE_ELIGIBILITY

"""

df_lounge_eligibility = icw.queryToDataframe(query)

In [ ]:
df_lounge_eligibility.head(2)

In [ ]:
df_lounge_eligibility.shape

In [ ]:
# a look to a particular flight number and date

#mask = (df_lounge_eligibility['DISCHARGE_STN_CD'] == 'GCM   ') & (df_lounge_eligibility['GMT_UPLIFT_DT'] == dt.date(2019,9,12) )

#df_lounge_eligibility[mask]

#### S19 Flight info

In [ ]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_FLIGHT_INFO

"""

df_flight_info = icw.queryToDataframe(query)

In [ ]:
df_flight_info.head(2)
df_flight_info.dtypes

#### Station Code Decode

In [ ]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_COUNTRY_DECODE

"""

df_country = icw.queryToDataframe(query)

In [ ]:
df_country.head(2)

#### Additional Station Decodes from ICW reference table

In [ ]:
# There are lots of missing destinations which are in lounge_elig but not in df_country.
# They are all new (since ~2019) arrivals to Heathrow.
# They are not in sandbox country dataset, but they are in a reference table on ICW.
# Load that ICW reference table and filter for results not in our merged table, but that are in lounge_elig.
# This should result in zero nan values.
query = """
SELECT STN_CD, COUNTRY_CD, COUNTRY_NM, CORP_GEOG_CTRY_GRP_NM, CORP_GEOG_CONTINENT_NM
FROM REF_GEOG_LOC_HIERARCHY
"""
df_additional_country_decodes=icw.queryToDataframe(query)
df_additional_country_decodes.shape # we will remove destinations already in df_country and not in df_lounge_elig later.

In [ ]:
df_additional_country_decodes.head(5)

#### Aircraft Type

In [ ]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_AC_TYPE

"""

df_acft_typ = icw.queryToDataframe(query)

In [ ]:
df_acft_typ.head(2)

In [ ]:
df_acft_typ.shape

In [ ]:
df_acft_typ.head(5)

<br>

### Pre-processing

*Hint:* It is always worth checking the format of each of the columns in your dataframes before trying to do any work with them. To do so, you can make use of the `headers_and_first_row` function below.


In [ ]:
def headers_and_first_row(df):
    '''
    print headers and first row of a df to deal with data types
    '''
    
    headers = df.columns
    first_row = []

    for col in headers:
        first_row.append(df[col][0])
    
    dictionary = dict( zip( headers, first_row) )

    return dictionary

In [ ]:
# applying headers_and_first_row to df_lounge_eligibility
format_df = headers_and_first_row(df_lounge_eligibility)
format_2 = headers_and_first_row(df_flight_info)

In [ ]:
# Note that some columns have blank spaces!
format_df
format_2

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 1:** Pre-process **all** the tables above (df_lounge_eligibility, df_flight_info, df_country, df_acft_typ).

</font>

</ol>
</div>

We will first strip all frames of spaces. We will then go through each frame to properly prepare it.

We will then load some additional datasets that will be useful for analysis.

In [ ]:
# Stripping strings

tables = [df_lounge_eligibility,df_flight_info,df_country,df_acft_typ,df_additional_country_decodes]

# Define a function that fine all string fields and remove all blak spaces
def data_cleaning_string(df):
    # Get names of all fields in a dataframe
    fields = df.columns
    # loop for all fields, if data type is string then remove blank spaces
    for f in fields:
        if type(df[f][0]) == str:
            df[f] = df[f].str.strip()
    return df

for i,t in enumerate(tables):
    tables[i] = data_cleaning_string(t)

In [ ]:
# Removing duplucates of flight_info

# Sort the table
df_flight_info = df_flight_info.sort_values(['GMT_PLND_DEP_TS','OPG_FLT_NO','GMT_ACT_DEP_TS'])
# Remove all duplicates and keep the first rows of all duplicated
df_flight_info = df_flight_info.drop_duplicates(subset = ['OPG_FLT_NO','GMT_PLND_DEP_TS'],keep = "first")


In [ ]:
# 2. Flight info
# Ian's code to clean dataset - identify latest departure date for
# duplicated flights and eliminate it.
df_flight_info['GMT_PLND_DEP_DT'] = df_flight_info['GMT_PLND_DEP_TS'].dt.date
df_flight_info['GMT_PLND_DEP_TIME'] = df_flight_info['GMT_PLND_DEP_TS'].dt.time

In [ ]:
# 3. Country
df_country['DEP_STN_CD'] = df_country['ROUTE'].str.slice(0,3)
df_country['ARR_STN_CD'] = df_country['ROUTE'].str.slice(3,6)


In [ ]:
# 4. Aircraft type
df_acft_typ.sort_values('FIRST_SEATS_QTY').tail(10)

In [ ]:
df_lounge_eligibility.shape

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 2:** Join the tables below
   
    - df_flight_info
    - df_country
    - df_acft_typ
    
to the table df_lounge_eligibility to generate a final dataset.

</font>

</ol>
</div>

In [ ]:
# df_lounge_eligibility.head(5)

In [ ]:
df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['OPERATING_AIRLINE_CD','OPERATING_FLT_NO','GMT_UPLIFT_DT'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['OPG_ALN_CD','OPG_FLT_NO','GMT_PLND_DEP_DT'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

df_lounge_elig_flight_info.head(5)

In [ ]:
# your code here!
df_lounge_elig_flight_info.shape

Get rows with any null values

In [ ]:
print(df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)].shape)
# There are currently 1183 rows for which flights are not available.

In [ ]:
# Example of missing flight number.
print(df_flight_info[df_flight_info['OPG_FLT_NO'] == 8642].head(5))
print(df_lounge_elig_flight_info[df_lounge_elig_flight_info['OPERATING_FLT_NO']==8642]['Skew_Id'].head(5))

### Merge `df_country` to table

In [ ]:
[headers_and_first_row(d) for d in [df_lounge_elig_flight_info,df_country]]

In [ ]:
df_additional_country_decodes.columns
df_country.columns

In [ ]:
# For efficiency, we should merge df_country and df_additional_country_decodes first.
# Firstly, organise df_additional_country_decodes to have the same columns.
# if statement to avoid throwing errors if column renaming from cell below has already been done.
if 'STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'STN_CD'
elif 'ARR_STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'ARR_STN_CD'
df_additional_country_decodes = df_additional_country_decodes[~df_additional_country_decodes[station_code_col].isin(df_country['ARR_STN_CD'])]
# for more efficiency, we can remove all those rows which are not needed as there are no lounge elig rows with that destinations.
df_additional_country_decodes = df_additional_country_decodes[df_additional_country_decodes[station_code_col].isin(df_lounge_elig_flight_info['DISCHARGE_STN_CD'])]
print(df_additional_country_decodes.shape)

In [ ]:
# We need df_additional_country_decodes to have the same column names and order as df_country, ready for pd.concat.
# By merging now, we eliminate the need to merge separately with df_lounge_elig.
df_additional_country_decodes['ROUTE'] = 'LHR' + df_additional_country_decodes['STN_CD']
df_additional_country_decodes['DEP_STN_CD'] = 'LHR'
# We must ensure column names and order of df_additional_country_decodes match df_country.
df_additional_country_decodes.rename(columns={'STN_CD':'ARR_STN_CD'},inplace=True)
df_additional_country_decodes = df_additional_country_decodes[df_country.columns]

In [ ]:
# concat together.
df_country = pd.concat([df_country,df_additional_country_decodes])

In [ ]:
df_lounge_elig_flight_info_country = pd.merge(df_lounge_elig_flight_info,# left table
                                     df_country, # right table
                                     left_on = ['DISCHARGE_STN_CD'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['ARR_STN_CD'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

# df_lounge_elig_flight_info = df_lounge_elig_flight_info.drop_duplicates()

df_lounge_elig_flight_info_country.head(5)

In [ ]:
df_lounge_elig_flight_info_country.shape

Check for null values

In [ ]:
df_lounge_elig_flight_info_country[df_lounge_elig_flight_info_country['CORP_GEOG_CONTINENT_NM'].isna()].head(5)
# There are no null values arising from lack of country data.

### `df_acft_typ`

In [ ]:
df_lounge_elig_flight_info_country_acft_typ = pd.merge(
    df_lounge_elig_flight_info_country,
    df_acft_typ,
    left_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    right_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    how = 'left'
)

df_lounge_elig_flight_info_country_acft_typ.head(5)

In [ ]:
df_lounge_elig_flight_info_country_acft_typ.shape

Check for null

In [ ]:
df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ.isna().any(axis=1)].head(5)

In [ ]:
df_acft_typ[df_acft_typ['ACT_AC_TYP_CD'] == 'M4']

### More cleaning for columns redundancy

In [ ]:
headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

In [ ]:
# drop redundant columns.

df_final = df_lounge_elig_flight_info_country_acft_typ.copy()

to_drop = ['GMT_PLND_DEP_TS','GMT_ACT_DEP_TS','OPG_ALN_CD','OPG_FLT_NO','ACT_DEP_STN_CD','PLND_ARR_STN_CD','ACT_ARR_STN_CD','GMT_UPLIFT_DT','ROUTE_x',
          'GMT_PLND_DEP_DT','GMT_PLND_DEP_TIME','ROUTE_y','DEP_STN_CD','ARR_STN_CD']

df_final = df_final.drop(columns = to_drop,)

df_final.head(5)

Save as CSV

In [ ]:
#df_final.to_csv('scragg_merged_df_final_2023-10-31-1330.csv')

<br><br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">Exercise 3: Based on S2019/S2023 data, provide a lookup table of Lounge eligibility assumptions that can be applied to a future schedule. To do so, answer each of the following questions in the Markdown cell provided below. 

- What level of granularity do you use?
- What metric do you use to come up with Lounge eligibility profiles?

    
Note 1: **Provide evidence for your assumptions.** This can be in the form of tables, graphs, correlation matrix, etc.
    
Note 2: Make use of the examples below to give structure to your answer. Feel free to attend the Hackathon Clinics if you have any questions. 
</font>


    
</ol>
</div>

**Reasoning (Example 1)**:


Assume:
- Data has been preprocessed.
- Data has been joined, and a final dataset has been created. This dataset is the result of joining the 4 tables.

The final dataset has been called `df_lounge_elig_flight_info_country_acft_typ`.

<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their aircraft type. I will therefore have a lookup table with two categories: Narrowbody and Widebody. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by Aircraft Type, by Tier. Then I have divided them by the total number of passengers by Aircraft Type. For example:
    - For NB aircrafts, and for Tier 1 passengers: In S19 we had 41,728 pax eligible for Tier 1 out of 7,222,830 pax flying on Narrowbody aircraft. This represents 0.6% of the costumers and I assume that this will be the number of costumers elegible for this specific Lounge in a future schedule.   

In [ ]:
# your code here!

In [ ]:
# columns that you want to group by
list_groupby = ['WB_NB_CAT','Lounge_eligibility_tier']

# grouping by WB_NB_CAT and Lounge_eligibility_tier
df_groupby_wb_nb = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

In [ ]:
# a look at the data
df_groupby_wb_nb

In [ ]:
# getting the number of pax by aircraft type

# columns that you want to group by
list_groupby = ['WB_NB_CAT']

# grouping by WB_NB_CAT
df_groupby_wb_nb_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [ ]:
# a look at the data
df_groupby_wb_nb_ttl

In [ ]:
# now, let's left join df_groupby_wb_nb_ttl onto df_groupby_wb_nb
# this adds a new column to the df_groupby_wb_nb table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_wb_nb = pd.merge(df_groupby_wb_nb,
                            df_groupby_wb_nb_ttl,
                            on = ['WB_NB_CAT'],
                            how = 'left'
                           )

In [ ]:
# a look at the data
df_groupby_wb_nb

In [ ]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_wb_nb['pax_eligible%'] = (df_groupby_wb_nb['pax_count'] / df_groupby_wb_nb['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_wb_nb['pax_eligible%'] = df_groupby_wb_nb['pax_eligible%'].map('{:,.1f}%'.format)

In [ ]:
# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_wb_nb.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [ ]:
# a look at the data
df_groupby_wb_nb

In [ ]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_wb_nb['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_wb_nb = df_groupby_wb_nb[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [ ]:
# using set_index to come up with the final lookup table
df_groupby_wb_nb = df_groupby_wb_nb.set_index(['WB_NB_CAT','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [ ]:
# final table
df_groupby_wb_nb

<br>

**Feedback:** This analysis provides a lookup table in the format needed to be input into a future schedule. Nevertheless, this analysis is too high level and you haven't provided any evidence for your assumptions. To further enhance your answer use insights from the data and provide evidence for your assumptions. Please find some ideas below: 

- Using the same categories (WB,NB), plot data overtime to better understand the peaks for the different lounges. 
- Is there a way to split Widebody into more categories? Do the Haul, Region, Time of Day, or Country play a role in the number of passengers that are eligible in Tier 1, Tier 2 and Tier 3? Etc...

<br><br>

**Reasoning (Example 2)**:



<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their flight number. I will therefore have a lookup table with a lot of categories as each flight number is a category. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by flight number, by Tier. Then I have divided them by the total number of passengers flight number.  

In [ ]:
# your code here!

In [ ]:
# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_flt_no = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_flt_no.head(4)

In [ ]:
# getting the number of pax by OPERATING_FLT_NO and DISCHARGE_STN_CD

# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD']

# grouping by WB_NB_CAT
df_groupby_flt_no_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [ ]:
# a look at the data
df_groupby_flt_no_ttl.head(2)

In [ ]:
# now, let's left join df_groupby_flt_no_ttl onto df_groupby_flt_no
# this adds a new column to the df_groupby_flt_no table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_flt_no = pd.merge(df_groupby_flt_no,
                            df_groupby_flt_no_ttl,
                            on = ['OPERATING_FLT_NO','DISCHARGE_STN_CD'],
                            how = 'left'
                           )

In [ ]:
# a look at the data
df_groupby_flt_no.head(4)

In [ ]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_flt_no['pax_eligible%'] = (df_groupby_flt_no['pax_count'] / df_groupby_flt_no['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_flt_no['pax_eligible%'] = df_groupby_flt_no['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_flt_no.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [ ]:
# a look at the data
df_groupby_flt_no.head(4)

In [ ]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_flt_no['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_flt_no = df_groupby_flt_no[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [ ]:
# using set_index to come up with the final lookup table
df_groupby_flt_no = df_groupby_flt_no.set_index(['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [ ]:
# final table
df_groupby_flt_no.head(4)

<br>

**Feedback:** This analysis goes at a very granular level, we might come up with missing values if we apply this lounge eligibility profiles to a future schedule. Here are some ideas to further enhance your answer: 

- What would happen if we fly to a new destination in the future? How do we ensure we have a lounge eligibility profile for this new route?
- As you can see in the example above: Pax eligible for Tier 1 for the BKK flight is significantly different from the rest. What's the most used aircraft type for this route? And why it differs that much from the rest? Is it because of the route characteristics instead?
- For SH routes, we might change the time of departure for a specific flight number from one year to the next. Explore the possibility of using a clasification that takes that into account, like using Time of Day instead of flight number. 

### Ian  
First of all I would like to be able to split the data into 2019/23 data separately, which can be done by adding a year field into the table.

In [ ]:
# Original merged table
headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

In [ ]:
# df_lounge_elig_flight_info_country_acft_typ['GMT_PLND_DEPT_YR'] = df_lounge_elig_flight_info_country_acft_typ['GMT_PLND_DEP_DT'].year

# headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)
df_lounge_elig_flight_info_country_acft_typ['GMT_UPLIFT_YR'] = [dts.year for dts in df_lounge_elig_flight_info_country_acft_typ['GMT_UPLIFT_DT']]

df_lounge_elig_flight_info_country_acft_typ.head(10)

In [ ]:
# columns that you want to group by
list_groupby = ['GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year

JAMIE

In [ ]:
# I want to get numbers by time of day, split by terminal.
# Numbers split long and short haul with lounge elig attached.

# add GMT Uplift Yr if needed

df_final_col_list = list(df_final.columns)
pivot_table_drop = ['BA_PAX_TIER','ONEWORLD_TIER','Lounge_eligibility_tier','pax']

pivot_df_final = pd.pivot_table(df_final, values='pax', index='Skew_Id',
                                columns='Lounge_eligibility_tier',aggfunc='sum',fill_value='NA').reset_index()
pivot_df_final.head()

In [ ]:
pivot_df_final = pivot_df_final.merge(df_final.drop(pivot_table_drop,axis=1),on='Skew_Id',how='left')

--

--

--

-

-

-

-

-

-


### Linear Regression

In [78]:
# import numpy as np
# from sklearn import datasets, linear_model
# import matplotlib.pyplot as plt

In [79]:
# headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

Separate the large tables to different tiers

In [80]:
# df_merged_T1 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 1'].reset_index()
# df_merged_T2 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 2'].reset_index()
# df_merged_T3 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 3'].reset_index()

# df_merged_T1 = df_merged_T1.rename(columns={'pax':'Tier_1_pax'})
# df_merged_T2 = df_merged_T2.rename(columns={'pax':'Tier_2_pax'})
# df_merged_T3 = df_merged_T3.rename(columns={'pax':'Tier_3_pax'})

In [81]:
# df_merged_T3.columns
# headers_and_first_row(df_merged_T1)


In [82]:
# y = np.array([1,2,3])
# x = np.array([-1,0,1])

# # y = y.to_numpy()
# # x = x.to_numpy().reshape(-1, 1)
# x = x.reshape(-1,1)

# # print(x.shape, y.shape)

# regr = linear_model.LinearRegression()
# regr.fit(x,y)

# regr.coef_, regr.intercept_, regr.rank_

### Compare by Year

In [83]:
# columns that you want to group by
list_groupby = ['GMT_UPLIFT_YR']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year_ttl

,GMT_UPLIFT_YR,pax_count_ttl
0,2019,10975714
1,2023,9737126


In [84]:
df_groupby_year = pd.merge(df_groupby_year,
                            df_groupby_year_ttl,
                            on = ['GMT_UPLIFT_YR'],
                            how = 'left'
                           )

df_groupby_year

,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,2019,Not eligible,8147159,10975714
1,2019,Tier 1,196564,10975714
2,2019,Tier 2,635765,10975714
3,2019,Tier 3,1996226,10975714
4,2023,Not eligible,7372440,9737126
5,2023,Tier 1,93555,9737126
6,2023,Tier 2,501044,9737126
7,2023,Tier 3,1770087,9737126


In [85]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_year['pax_eligible%'] = (df_groupby_year['pax_count'] / df_groupby_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_year['pax_eligible%'] = df_groupby_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [86]:
df_groupby_year = df_groupby_year[df_groupby_year['Lounge_eligibility_tier'] != "Not eligible"]

df_groupby_year

,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_eligible%
1,2019,Tier 1,1.8%
2,2019,Tier 2,5.8%
3,2019,Tier 3,18.2%
5,2023,Tier 1,1.0%
6,2023,Tier 2,5.1%
7,2023,Tier 3,18.2%


Not much difference

### Compare by location and year

In [88]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_country_group_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR']

df_groupby_country_group_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_country_group_year = pd.merge(df_groupby_country_group_year,
                                         df_groupby_country_group_year_ttl,
                                         on = list_groupby,
                                         how = "left")

df_groupby_country_group_year = df_groupby_country_group_year[df_groupby_country_group_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_country_group_year['pax_eligible%'] = (df_groupby_country_group_year['pax_count'] / df_groupby_country_group_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_country_group_year['pax_eligible%'] = df_groupby_country_group_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_country_group_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_country_group_year.head(6)


,CORP_GEOG_CONTINENT_NM,CORP_GEOG_CTRY_GRP_NM,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_eligible%
1,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 1,3.7%
2,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 2,4.0%
3,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 3,17.6%
5,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 1,0.2%
6,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 2,4.3%
7,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 3,16.0%


In [89]:
df_groupby_country_group_year = df_groupby_country_group_year.set_index(['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_country_group_year

pax_eligible%  \
Lounge_eligibility_tier                                                  Tier 1   
CORP_GEOG_CONTINENT_NM      CORP_GEOG_CTRY_GRP_NM   GMT_UPLIFT_YR                 
AFRICA                      EAST AND CENTRAL AFRICA 2019                   3.7%   
                                                    2023                   0.2%   
                            NORTH AFRICA            2019                   0.5%   
                                                    2023                   0.3%   
                            SOUTHERN AFRICA         2019                   3.8%   
                                                    2023                   2.4%   
                            WEST AFRICA             2019                   3.7%   
                                                    2023                   2.3%   
ASIA FAR EAST               CHINA                   2019                   4.6%   
                                                    2023                   0.2%   
                            FAR EAST HUBS           2019                   3.8%   
                                                    2023                   1.4%   
                            FAR EAST OTHER          2019                   0.4%   
                            JAPAN                   2019                   3.5%   
                                                    2023                   1.0%   
ASIA SOUTH EAST AND CENTRAL SOUTH ASIA              2019                   2.8%   
                                                    2023                   0.4%   
AUSTRALASIA PACIFIC         FAR EAST JSA HUBS       2019                   2.3%   
                                                    2023                   3.4%   
EUROPE EXC UK               BENELUX                 2019                   0.7%   
                                                    2023                   0.5%   
                            EASTERN EUROPE          2019                   0.5%   
                                                    2023                   0.2%   
                            IRELAND RO              2019                   0.6%   
                                                    2023                   0.6%   
                            MEDITERRANEAN           2019                   0.7%   
                                                    2023                   0.3%   
                            SCANDINAVIA             2019                   0.6%   
                                                    2023                   0.4%   
                            WEST EUROPE             2019                   0.5%   
                                                    2023                   0.4%   
MIDDLE EAST                 MIDDLE EAST OTHER       2019                   0.3%   
                                                    2023                   0.1%   
                            NORTHERN GULF           2019                   5.0%   
                                                    2023                   3.0%   
                            SAUDI ARABIA            2019                   4.0%   
                                                    2023                   3.5%   
                            SOUTHERN GULF           2019                   4.8%   
                                                    2023                   3.6%   
NORTH AMERICA               CANADA                  2019                   2.3%   
                                                    2023                   0.2%   
                            NORTH AMERICA OTHER     2023                   3.6%   
                            USA                     2019                   4.3%   
                                                    2023                   2.4%   
SOUTH AMERICA INC CARIBBEAN CARIBBEAN               2019                    NaN   
                                                    2023                   2.7%   

### By continent

In [90]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_cont_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR']

df_groupby_cont_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_cont_year = pd.merge(df_groupby_cont_year,
                                df_groupby_cont_year_ttl,
                                on = list_groupby,
                                how = "left")

df_groupby_cont_year = df_groupby_cont_year[df_groupby_cont_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_cont_year['pax_eligible%'] = (df_groupby_cont_year['pax_count'] / df_groupby_cont_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_cont_year['pax_eligible%'] = df_groupby_cont_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_cont_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_cont_year.head(6)

,CORP_GEOG_CONTINENT_NM,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_eligible%
1,AFRICA,2019,Tier 1,3.5%
2,AFRICA,2019,Tier 2,4.8%
3,AFRICA,2019,Tier 3,18.5%
5,AFRICA,2023,Tier 1,1.7%
6,AFRICA,2023,Tier 2,4.3%
7,AFRICA,2023,Tier 3,18.2%


In [91]:
df_groupby_cont_year = df_groupby_cont_year.set_index(['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_cont_year

pax_eligible%              
Lounge_eligibility_tier                          Tier 1 Tier 2 Tier 3
CORP_GEOG_CONTINENT_NM      GMT_UPLIFT_YR                            
AFRICA                      2019                   3.5%   4.8%  18.5%
                            2023                   1.7%   4.3%  18.2%
ASIA FAR EAST               2019                   3.6%   4.8%  18.3%
                            2023                   1.0%   3.5%  18.3%
ASIA SOUTH EAST AND CENTRAL 2019                   2.8%   4.0%  19.1%
                            2023                   0.4%   3.0%  17.0%
AUSTRALASIA PACIFIC         2019                   2.3%   5.6%  18.3%
                            2023                   3.4%   7.4%  26.3%
EUROPE EXC UK               2019                   0.6%   6.4%  17.1%
                            2023                   0.4%   5.5%  18.1%
MIDDLE EAST                 2019                   4.0%   6.0%  20.2%
                            2023                   3.1%   5.0%  18.8%
NORTH AMERICA               2019                   4.1%   5.2%  20.5%
                            2023                   2.3%   5.1%  19.2%
SOUTH AMERICA INC CARIBBEAN 2019                   2.4%   3.6%  17.4%
                            2023                   1.5%   3.1%  17.3%
UK                          2019                   0.4%   5.7%  18.1%
                            2023                   0.4%   5.0%  16.8%

### By aircraft

In [92]:
list_groupby = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD','FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_acft = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                mean_pax_count = ('pax','mean')).reset_index()

df_groupby_acft['mean_pax_count'] = df_groupby_acft['mean_pax_count'].map('{:,.1f}'.format)

df_groupby_acft = df_groupby_acft[df_groupby_acft['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_acft = df_groupby_acft.set_index(list_groupby,drop = True).unstack('Lounge_eligibility_tier')

df_groupby_acft.fillna(0, inplace=True)

df_groupby_acft


mean_pax_count  \
Lounge_eligibility_tier                                                                                      Tier 1   
IATA_AC_TYP_CD ACT_AC_TYP_CD FIRST_SEATS_QTY CLUB_SEATS_QTY PREM_ECONOMY_SEATS_QTY ECONOMY_SEATS_QTY                  
319            A4            0.0             16.0           0.0                    119.0                        1.2   
               M4            0.0             21.0           0.0                    113.0                        1.1   
320            A3            0.0             24.0           0.0                    132.0                        1.2   
               E9            0.0             29.0           0.0                    130.0                        1.2   
               M2            0.0             24.0           0.0                    144.0                        1.1   
               R3            0.0             28.0           0.0                    139.0                        1.2   
               T3            0.0             26.0           0.0                    142.0                        1.2   
321            M6            0.0             23.0           0.0                    131.0                        1.3   
               V6            0.0             38.0           0.0                    160.0                        1.2   
               W6            0.0             42.0           0.0                    154.0                        1.3   
32A            H3            0.0             20.0           0.0                    150.0                        1.2   
32N            N3            0.0             20.0           0.0                    150.0                        1.2   
32Q            N6            0.0             32.0           0.0                    172.0                        1.2   
332            W9            0.0             30.0           21.0                   235.0                        1.0   
339            W9            0.0             30.0           21.0                   234.0                        1.1   
343            W9            0.0             45.0           0.0                    212.0                        1.3   
351            K5            0.0             56.0           56.0                   219.0                        1.3   
388            A8            14.0            97.0           55.0                   303.0                        2.2   
744            S4            14.0            86.0           30.0                   145.0                        2.1   
               V4            14.0            52.0           36.0                   235.0                        2.1   
777            D7            0.0             48.0           40.0                   184.0                        1.3   
               J7            0.0             32.0           48.0                   252.0                          0   
               K7            0.0             48.0           24.0                   203.0                          0   
               L7            0.0             48.0           24.0                   203.0                        1.3   
               M7            7.0             50.0           40.0                   138.0                        1.6   
               P7            17.0            48.0           24.0                   127.0                        2.1   
               R7            10.0            52.0           40.0                   134.0                        1.7   
               V7            14.0            48.0           40.0                   124.0                        2.1   
               W7            12.0            48.0           32.0                   127.0                        2.0   
77W            F7            8.0             76.0           40.0                   130.0                        1.7   
               G7            14.0            56.0           44.0                   183.0                        2.1   
781            X8            7.0             49.0      

<br><br><br>

In [93]:
#--------------------------------
#
# Your turn!!!
#
#--------------------------------

<u>What level of granularity do you use?</u>
- ... (your answer here)

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- ... (your answer here)

In [ ]:
# your code here

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.